In [2]:
########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
np.random.seed(1337)

import pandas as pd
import operator
import sys

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from iwillwin.trainer.supervised_trainer import KerasModelTrainer
from iwillwin.data_utils.data_helpers import DataTransformer, DataLoader, CharDataTransformer
from iwillwin.model.sim_zoos import *
import tensorflow as tf
from keras.layers import Dense, Input, MaxPooling1D, CuDNNLSTM, Embedding, Add, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from iwillwin.config import dataset_config, model_config
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.core import Lambda, Dense, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.legacy.layers import Highway
from keras.layers import TimeDistributed
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import log_loss

Using TensorFlow backend.
C:\Users\zake7\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zake7\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
NB_WORDS = 5000
EMBEDDING_DIM = 150
MAX_SEQUENCE_LENGTH = 50
OUT_SIZE = 1

In [9]:
data_transformer = CharDataTransformer(max_num_words=NB_WORDS, max_sequence_length=MAX_SEQUENCE_LENGTH, char_level=False,
                                   normalization=True, features_processed=True)
trains, tests, labels = data_transformer.prepare_data(dual=False)
print("Number of unique words", len(data_transformer.tokenizer.index_docs))
labels = to_categorical(labels)

[DataHelper] Apply normalization on value-type columns


C:\Users\zake7\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Doing preprocessing...
Transforming words to indices...
Shape of data tensor: (320552, 50) (320552, 50)
Shape of label tensor: (320552,)
Preprocessed.
Number of unique words 5237


## Load and prepare the data

In [4]:
print("Embeddings")
print(os.listdir("../data/wordvec"))

Embeddings
['.gitkeep', 'fasttext-50-win3.vec', 'zh-wordvec-50-cbow-windowsize50.vec', 'zh-wordvec-50-skipgram-windowsize7.vec']


In [5]:
data_loader = DataLoader()
skip_gram_embeddings = data_loader.load_embedding('../data/wordvec/zh-wordvec-50-skipgram-windowsize7.vec')
cbow_embeddings = data_loader.load_embedding('../data/wordvec/zh-wordvec-50-cbow-windowsize50.vec')
fasttext_embeddings = data_loader.load_embedding('../data/wordvec/fasttext-50-win3.vec')

Total 4114 word vectors.
Total 4114 word vectors.
Total 4114 word vectors.


In [6]:
def build_embedding_matrix(embeddings_index, nb_words=NB_WORDS, word_index=data_transformer.tokenizer.word_index):
    #nb_words = min(nb_words, len(embeddings_index))
    #embedding_matrix = np.random.rand(nb_words, 50)
    embedding_matrix = np.zeros((nb_words, 50))
    
    word_index = data_transformer.tokenizer.word_index
    null_words = open('null-word.txt', 'w', encoding='utf-8')

    for word, i in word_index.items():
        if i >= nb_words:
            null_words.write(word + '\n')
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            null_words.write(word + '\n')
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix

In [7]:
cbow_matrix = build_embedding_matrix(cbow_embeddings)
skipgram_matrix = build_embedding_matrix(skip_gram_embeddings)
fasttext_matrix = build_embedding_matrix(fasttext_embeddings)

Null word embeddings: 948
Null word embeddings: 948
Null word embeddings: 948


In [8]:
meta_embeddings = np.concatenate((cbow_matrix, skipgram_matrix, fasttext_matrix), axis=1)

In [9]:
meta_embeddings[0] = np.array([0] * 150) # zero padding

# Add tricky features

## Rumor words

In [10]:
train_df = pd.read_csv('../data/dataset/train.csv')
test_df = pd.read_csv('../data/dataset/test.csv')

In [11]:
rumor_words = ['辟谣', '谣言', '勿传', '假的']

def is_rumor(text):
    if type(text) != str:
        print(text, type(text))
        return 0
    for rumor_word in rumor_words:
        if rumor_word in text:
            return 1
    return 0

def has_split_symbol(text):
    if type(text) != str:
        return 0
    if '|' in text:
        return 1
    return 0

for df in [train_df, test_df]:
    df['has_|'] = df['title2_zh'].apply(has_split_symbol)
    df['has_rumor_words'] = df['title2_zh'].apply(is_rumor)

nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>
nan <class 'float'>


In [12]:
train_has_rumor = train_df.has_rumor_words.values
test_has_rumor = test_df.has_rumor_words.values

trick_trains_features = np.concatenate((trains[2], train_has_rumor.reshape((-1, 1))), axis=1)
trick_tests_features = np.concatenate((tests[2], test_has_rumor.reshape((-1, 1))), axis=1)

## Exact Match

In [13]:
def _build_exact_match_sequences(sent_1, sent_2):
    sent_1_char_set = set(sent_1)
    sent_2_char_set = set(sent_2)
    intersection = sent_1_char_set & sent_2_char_set
    
    sent_1_em = np.zeros_like(sent_1)
    sent_2_em = np.zeros_like(sent_2)

    for i in range(len(sent_1)):
        if sent_1[i] == 0:
            continue
        if sent_1[i] in intersection:
            sent_1_em[i] = 1
    
    for i in range(len(sent_2)):
        if sent_2[i] == 0:
            continue        
        if sent_2[i] in intersection:
            sent_2_em[i] = 1
    
    return sent_1_em, sent_2_em

def build_exact_match_sequences(sents_1, sents_2):
    sents_1_em, sents_2_em = [], []
    for sent_1, sent_2 in zip(sents_1, sents_2):
        sent_1_em, sent_2_em = _build_exact_match_sequences(sent_1, sent_2)
        sents_1_em.append(sent_1_em)
        sents_2_em.append(sent_2_em)
    return np.array(sents_1_em), np.array(sents_2_em)

In [14]:
%%time
trains_1_ems, trains_2_ems = build_exact_match_sequences(trains[0], trains[1])
tests_1_ems, tests_2_ems = build_exact_match_sequences(tests[0], tests[1])

Wall time: 15.1 s


In [15]:
print("Shape of train em", trains_1_ems.shape, trains_2_ems.shape)
print("Shape of test em", tests_1_ems.shape, tests_2_ems.shape)

Shape of train em (320552, 50) (320552, 50)
Shape of test em (80126, 50) (80126, 50)


In [16]:
em_train_features = (trains_1_ems, trains_2_ems)
em_test_features = (tests_1_ems, tests_2_ems)

In [17]:
#class_weight.compute_class_weight('balanced', [0, 1, 2], labels.argmax(axis=1))

# Trick or Treat!!!

In [18]:
use_tricky = True

if use_tricky:
    trains = (trains[0], trains[1], trick_trains_features)
    tests = (tests[0], tests[1], trick_tests_features)
else:
    trains = (trains[0], trains[1], trains[2])
    tests = (tests[0], tests[1], tests[2])

In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import importlib

from sklearn.metrics import roc_auc_score, log_loss
from keras.callbacks import EarlyStopping, ModelCheckpoint

from iwillwin.config import model_config

class ModelTrainer(object):

    def __init__(self, model_stamp, epoch_num, learning_rate=1e-3,
                 shuffle_inputs=False, verbose_round=40, early_stopping_round=8):
        self.models = []
        self.model_stamp = model_stamp
        self.val_loss = -1
        self.auc = -1
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.eps = 1e-10
        self.verbose_round = verbose_round
        self.early_stopping_round = early_stopping_round
        self.shuffle_inputs = shuffle_inputs
        self.class_weight = [0.93, 1.21]

    def train_folds(self, X, y, fold_count, em_train_features, batch_size, get_model_func, augments=None, skip_fold=0, patience=10, scale_sample_weight=False,
                    class_weight=None, self_aware=False, swap_input=False):
        X1, X2, features, = X
        em1, em2 = em_train_features
        features = features
        #features = features[:, -1]
        weight_val=scale_sample_weight

        fold_size = len(X1) // fold_count
        models = []
        fold_predictions = []
        score = 0

        for fold_id in range(0, fold_count):
            fold_start = fold_size * fold_id
            fold_end = fold_start + fold_size

            if fold_id == fold_count - 1:
                fold_end = len(X1)

            train_x1 = np.concatenate([X1[:fold_start], X1[fold_end:]])
            train_x2 = np.concatenate([X2[:fold_start], X2[fold_end:]])
            train_features = np.concatenate([features[:fold_start], features[fold_end:]])
            
            train_em_1 = np.concatenate([em1[:fold_start], em1[fold_end:]])
            train_em_2 = np.concatenate([em2[:fold_start], em2[fold_end:]])
            
            train_y = np.concatenate([y[:fold_start], y[fold_end:]])

            val_x1 = X1[fold_start:fold_end]
            val_x2 = X2[fold_start:fold_end]
            val_features = features[fold_start:fold_end]
            val_em1 = em1[fold_start:fold_end]
            val_em2 = em2[fold_start:fold_end]
            val_y = y[fold_start:fold_end]

            fold_pos = (np.sum(train_y) / len(train_x1))

            train_data = {
                "first_sentences": train_x1,
                "second_sentences": train_x2,
                "mata-features": train_features,
                "first_exact_match": train_em_1,
                "second_exact_match": train_em_2,
            }

            val_data = {
                "first_sentences": val_x1,
                "second_sentences": val_x2,
                "mata-features": val_features,
                "first_exact_match": val_em1,
                "second_exact_match": val_em2,
            }

            model, bst_val_score, fold_prediction = self._train_model_by_logloss(
                get_model_func(), batch_size, train_data, train_y, val_data, val_y, fold_id, patience, class_weight, weight_val=None)
    
            score += bst_val_score
            models.append(model)
            fold_predictions.append(fold_prediction)

        self.models = models
        self.val_loss = score / fold_count
        return models, self.val_loss, fold_predictions

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience):
        # return a list which holds [models, val_loss, auc, prediction]
        raise NotImplementedError

class KerasModelTrainer(ModelTrainer):

    def __init__(self, *args, **kwargs):
        super(KerasModelTrainer, self).__init__(*args, **kwargs)
        pass

    def _train_model_by_logloss(self, model, batch_size, train_x, train_y, val_x, val_y, fold_id, patience, class_weight, weight_val):
        early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
        bst_model_path = self.model_stamp + str(fold_id) + '.h5'
        val_data = (val_x, val_y, weight_val) if weight_val is not None else (val_x, val_y)
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
        hist = model.fit(train_x, train_y,
                         validation_data=val_data,
                         epochs=self.epoch_num, batch_size=batch_size, shuffle=True,
                         callbacks=[early_stopping, model_checkpoint],
                         class_weight=class_weight)
        bst_val_score = max(hist.history['val_weighted_accuracy']) # note this is the hard version
        model.load_weights(bst_model_path)
        predictions = model.predict(val_x)

        return model, bst_val_score, predictions

In [20]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = tf.nn.leaky_relu(K.conv1d(u_vecs, self.W))
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = K.tanh(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)
    
def get_padding_mask(q, k):
    ones = K.expand_dims(K.ones_like(q, 'float32'), -1)
    mask = K.cast(K.expand_dims(K.not_equal(k, 0), 1), 'float32')
    mask = K.batch_dot(ones, mask, axes=[2,1])
    return mask

def unchanged_shape(input_shape):
    "Function for Lambda layer"
    return input_shape

def substract(input_1, input_2):
    "Substract element-wise"
    out_ = Lambda(lambda x: x[0] - x[1])([input_1, input_2])
    return out_

def eldistance(input_1, input_2):
    "Substract element-wise"
    out_ = Lambda(lambda x: K.sqrt(K.square(x[0] - x[1])))([input_1, input_2])
    return out_

def submult(input_1, input_2):
    "Get multiplication and subtraction then concatenate results"
    mult = Multiply()([input_1, input_2])
    add = Add()([input_1, input_2])
    sub = substract(input_1, input_2)
    distance = eldistance(input_1, input_2)
    
    dual = Concatenate()([input_1, input_2])
    dual = Dense(32, activation='relu')(dual)
    dual = Dropout(0.5)(dual)
    dual = Dense(8, activation='relu')(dual)
    
    out_= Concatenate()([sub, mult, ])
    return out_

def apply_multiple(input_, layers):
    "Apply layers to input then concatenate result"
    if not len(layers) > 1:
        raise ValueError('Layers list should contain more than 1 layer')
    else:
        agg_ = []
        for layer in layers:
            agg_.append(layer(input_))
        out_ = Concatenate()(agg_)
    return out_

def time_distributed(input_, layers):
    "Apply a list of layers in TimeDistributed mode"
    out_ = []
    node_ = input_
    for layer_ in layers:
        node_ = TimeDistributed(layer_)(node_)
    out_ = node_
    return out_

def soft_attention_alignment(input_1, input_2):
    "Align text representation with neural soft attention"
    attention = Dot(axes=-1)([input_1, input_2])
    w_att_1 = Lambda(lambda x: softmax(x, axis=1),
                     output_shape=unchanged_shape)(attention)
    w_att_2 = Permute((2, 1))(Lambda(lambda x: softmax(x, axis=2),
                             output_shape=unchanged_shape)(attention))
    in1_aligned = Dot(axes=1)([w_att_1, input_1])
    in2_aligned = Dot(axes=1)([w_att_2, input_2])
    return in1_aligned, in2_aligned    

def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

def get_dense_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q1_c = Input(shape=(max_sequence_length, 11), name='first_sentences_char')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q2_c = Input(shape=(max_sequence_length, 11), name='second_sentences_char')
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")

    embedding = Embedding(nb_words,
                            embedding_dim,
                            input_length=max_sequence_length,
                            trainable=True)
    
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.5)(q1_embed)
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.5)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    
    q1_encoded = th(q1_embed,)    
    q2_encoded = th(q2_embed,)
    
    q1_aligned, q2_aligned = soft_attention_alignment(q1_encoded, q2_encoded)
    q1_encoded = Concatenate()([q2_aligned, q1_encoded])
    q2_encoded = Concatenate()([q1_aligned, q2_encoded])  
    
    cnn_init = Conv1D(32, 1, strides=1, padding='same', activation='relu')
    q1_seq = cnn_init(q1_encoded)
    q2_seq = cnn_init(q2_encoded)
    
    cnns = [Conv1D(1, 3, strides=1, padding='same', activation='relu') for i in range(10)]
    
    for idx, cnn in enumerate(cnns):
        q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
        q1_encoded = Concatenate()([q1_seq, q2_aligned, q1_encoded])
        q2_encoded = Concatenate()([q2_seq, q1_aligned, q2_encoded])            
        q1_seq = cnn(q1_encoded)
        q2_seq = cnn(q2_encoded)    
    
    
    capsule_pooling = Capsule(num_capsule=8, dim_capsule=100, routings=3, share_weights=True)
    
    # Pooling
    q1_rep = Flatten()(capsule_pooling(q1_encoded))
    q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    
    #q1_rep = apply_multiple(q1_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(),])
    #q2_rep = apply_multiple(q2_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(),])    
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi, ])
    h_all = Dropout(0.5)(h_all)
    h_all = Dense(256, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, input_layer_3], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy'])
    model.summary()
    return model

In [23]:
from keras import regularizers

def get_darnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")

    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
 
    em_embeddings = Embedding(2, 1,
                     input_length=max_sequence_length,
                     trainable=True)   
    
    q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match)])
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    
    q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match)])
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    th = TimeDistributed(Highway(activation='relu'))
    #q1_embed = Dropout(0.2)(th(q1_embed,))
    #q2_embed = Dropout(0.2)(th(q2_embed,))    
    
    rnns = [CuDNNGRU(42, return_sequences=True) for i in range(3)]
    
    q1_res = []
    q2_res = []
    
    
    for idx, rnn in enumerate(rnns):
        q1_seq = rnn(q1_embed)
        q1_seq = Dropout(0.1)(q1_seq)
        q2_seq = rnn(q2_embed)
        q2_seq = Dropout(0.1)(q2_seq)
        q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
        
        q1_res.append(q2_aligned)
        q1_res.append(q1_seq)
        
        q2_res.append(q1_aligned)
        q2_res.append(q2_seq)
        
        q1_embed = Concatenate()([q1_seq, q2_aligned, q1_embed])
        q2_embed = Concatenate()([q2_seq, q1_aligned, q2_embed])            
        
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    q1_res = Concatenate()(q1_res)
    q2_res = Concatenate()(q2_res)
    
    q1_rep = apply_multiple(q1_res, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_res, [GlobalAvgPool1D(), GlobalMaxPool1D()])    
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.1)(h_all)
    h_all = Dense(256, activation='relu')(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6,), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [24]:
def weighted_accuracy(y_true, y_pred):
    weight = np.array([[1/16, 1/15, 1/5]])
    norm = [(1/16) + (1/15) + (1/5)]
    weight_mask = weight * y_true
    
    y_pred = K.cast(y_pred > 0.5, 'int32') # hard version
    y_true = K.cast(y_true, 'int32')
    
    res = K.cast(K.equal(y_pred, y_true), 'float32') * weight_mask / K.sum(weight_mask)
    res = K.sum(res)
    return res

In [25]:
model_manager = ModelManager()

# DenseRNN

In [48]:
fold_count = 8
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_darnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=128,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=25)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "3Embedding-3LayersDenseRNN42-Drop01-NoMeta-NoClassWeighted-WithEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2][:, -1],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4


C:\Users\zake7\Anaconda3\lib\site-packages\keras\legacy\layers.py:659: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
first_exact_match (InputLayer)  (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
second_exact_match (InputLayer) (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 112s 401us/step - loss: 0.3552 - acc: 0.8346 - weighted_accuracy: 0.8042 - val_loss: 0.3393 - val_acc: 0.8371 - val_weighted_accuracy: 0.8209
Epoch 2/500
280483/280483 [==============================] - 109s 390us/step - loss: 0.3097 - acc: 0.8586 - weighted_accuracy: 0.8335 - val_loss: 0.3648 - val_acc: 0.8267 - val_weighted_accuracy: 0.8124
Epoch 3/500
280483/280483 [==============================] - 113s 401us/step - loss: 0.2950 - acc: 0.8669 - weighted_accuracy: 0.8434 - val_loss: 0.3413 - val_acc: 0.8340 - val_weighted_accuracy: 0.8221
Epoch 4/500
280483/280483 [==============================] - 110s 393us/step - loss: 0.2862 - acc: 0.8719 - weighted_accuracy: 0.8488 - val_loss: 0.3227 - val_acc: 0.8504 - val_weighted_accuracy: 0.8402
Epoch 5/500
280483/280483 [==============================] - 110s 391us/step - loss: 0.2800 - acc: 0.8751 - weighted_accuracy: 0.8523 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 112s 401us/step - loss: 0.3561 - acc: 0.8350 - weighted_accuracy: 0.8040 - val_loss: 0.3175 - val_acc: 0.8557 - val_weighted_accuracy: 0.8267
Epoch 2/500
280483/280483 [==============================] - 111s 395us/step - loss: 0.3110 - acc: 0.8577 - weighted_accuracy: 0.8327 - val_loss: 0.3032 - val_acc: 0.8641 - val_weighted_accuracy: 0.8487
Epoch 3/500
280483/280483 [==============================] - 111s 396us/step - loss: 0.2976 - acc: 0.8652 - weighted_accuracy: 0.8409 - val_loss: 0.3081 - val_acc: 0.8584 - val_weighted_accuracy: 0.8464
Epoch 4/500
280483/280483 [==============================] - 111s 395us/step - loss: 0.2881 - acc: 0.8705 - weighted_accuracy: 0.8471 - val_loss: 0.3013 - val_acc: 0.8609 - val_weighted_accuracy: 0.8476
Epoch 5/500
280483/280483 [==============================] - 111s 396us/step - loss: 0.2820 - acc: 0.8739 - weighted_accuracy: 0.8512 - v

...

Epoch 41/500
280483/280483 [==============================] - 111s 397us/step - loss: 0.2376 - acc: 0.8969 - weighted_accuracy: 0.8774 - val_loss: 0.3136 - val_acc: 0.8562 - val_weighted_accuracy: 0.8410
Epoch 42/500
280483/280483 [==============================] - 111s 397us/step - loss: 0.2379 - acc: 0.8967 - weighted_accuracy: 0.8772 - val_loss: 0.3218 - val_acc: 0.8545 - val_weighted_accuracy: 0.8387
Epoch 43/500
280483/280483 [==============================] - 111s 396us/step - loss: 0.2367 - acc: 0.8974 - weighted_accuracy: 0.8781 - val_loss: 0.3277 - val_acc: 0.8506 - val_weighted_accuracy: 0.8373
Epoch 44/500
280483/280483 [==============================] - 111s 396us/step - loss: 0.2364 - acc: 0.8978 - weighted_accuracy: 0.8786 - val_loss: 0.3227 - val_acc: 0.8518 - val_weighted_accuracy: 0.8395
Epoch 45/500
280483/280483 [==============================] - 111s 397us/step - loss: 0.2368 - acc: 0.8974 - weighted_accuracy: 0.8779 - val_loss: 0.3231 - val_acc: 0.8535 - val_weight

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 113s 404us/step - loss: 0.3510 - acc: 0.8365 - weighted_accuracy: 0.8063 - val_loss: 0.3653 - val_acc: 0.8270 - val_weighted_accuracy: 0.8006
Epoch 2/500
280483/280483 [==============================] - 112s 398us/step - loss: 0.3051 - acc: 0.8607 - weighted_accuracy: 0.8353 - val_loss: 0.3511 - val_acc: 0.8334 - val_weighted_accuracy: 0.8116
Epoch 3/500
280483/280483 [==============================] - 111s 397us/step - loss: 0.2898 - acc: 0.8698 - weighted_accuracy: 0.8461 - val_loss: 0.3359 - val_acc: 0.8430 - val_weighted_accuracy: 0.8193
Epoch 4/500
280483/280483 [==============================] - 111s 397us/step - loss: 0.2820 - acc: 0.8742 - weighted_accuracy: 0.8506 - val_loss: 0.3371 - val_acc: 0.8429 - val_weighted_accuracy: 0.8124
Epoch 5/500
280483/280483 [==============================] - 111s 397us/step - loss: 0.2749 - acc: 0.8777 - weighted_accuracy: 0.8547 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 114s 405us/step - loss: 0.3540 - acc: 0.8354 - weighted_accuracy: 0.8046 - val_loss: 0.3393 - val_acc: 0.8415 - val_weighted_accuracy: 0.8268
Epoch 2/500
280483/280483 [==============================] - 112s 399us/step - loss: 0.3091 - acc: 0.8587 - weighted_accuracy: 0.8337 - val_loss: 0.3114 - val_acc: 0.8584 - val_weighted_accuracy: 0.8395
Epoch 3/500
280483/280483 [==============================] - 112s 399us/step - loss: 0.2960 - acc: 0.8664 - weighted_accuracy: 0.8424 - val_loss: 0.3065 - val_acc: 0.8579 - val_weighted_accuracy: 0.8431
Epoch 4/500
280483/280483 [==============================] - 112s 399us/step - loss: 0.2865 - acc: 0.8714 - weighted_accuracy: 0.8480 - val_loss: 0.3131 - val_acc: 0.8554 - val_weighted_accuracy: 0.8397
Epoch 5/500
280483/280483 [==============================] - 112s 400us/step - loss: 0.2802 - acc: 0.8754 - weighted_accuracy: 0.8524 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 114s 408us/step - loss: 0.3581 - acc: 0.8331 - weighted_accuracy: 0.8034 - val_loss: 0.3792 - val_acc: 0.8137 - val_weighted_accuracy: 0.8001
Epoch 2/500
280483/280483 [==============================] - 112s 400us/step - loss: 0.3113 - acc: 0.8579 - weighted_accuracy: 0.8336 - val_loss: 0.3087 - val_acc: 0.8546 - val_weighted_accuracy: 0.8242
Epoch 3/500
280483/280483 [==============================] - 112s 400us/step - loss: 0.2969 - acc: 0.8664 - weighted_accuracy: 0.8429 - val_loss: 0.3083 - val_acc: 0.8610 - val_weighted_accuracy: 0.8352
Epoch 4/500
280483/280483 [==============================] - 112s 400us/step - loss: 0.2891 - acc: 0.8703 - weighted_accuracy: 0.8472 - val_loss: 0.3101 - val_acc: 0.8556 - val_weighted_accuracy: 0.8453
Epoch 5/500
280483/280483 [==============================] - 112s 400us/step - loss: 0.2818 - acc: 0.8741 - weighted_accuracy: 0.8518 - v

Epoch 41/500
280483/280483 [==============================] - 110s 393us/step - loss: 0.2409 - acc: 0.8950 - weighted_accuracy: 0.8765 - val_loss: 0.2874 - val_acc: 0.8749 - val_weighted_accuracy: 0.8573
Epoch 42/500
280483/280483 [==============================] - 110s 394us/step - loss: 0.2405 - acc: 0.8957 - weighted_accuracy: 0.8767 - val_loss: 0.2956 - val_acc: 0.8744 - val_weighted_accuracy: 0.8546
Epoch 43/500
280483/280483 [==============================] - 110s 393us/step - loss: 0.2405 - acc: 0.8954 - weighted_accuracy: 0.8763 - val_loss: 0.2984 - val_acc: 0.8721 - val_weighted_accuracy: 0.8519
Epoch 44/500
280483/280483 [==============================] - 111s 394us/step - loss: 0.2401 - acc: 0.8954 - weighted_accuracy: 0.8763 - val_loss: 0.2901 - val_acc: 0.8740 - val_weighted_accuracy: 0.8563
Epoch 45/500
280483/280483 [==============================] - 110s 394us/step - loss: 0.2403 - acc: 0.8953 - weighted_accuracy: 0.8762 - val_loss: 0.2855 - val_acc: 0.8761 - val_weight

# DenseCNN

In [25]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

def get_dense_cnn(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    meta_features_input = Input(shape=(36,), name='mata-features')
    
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
    
    flex_embedding = Embedding(nb_words, 20,
                      input_length=max_sequence_length,
                      trainable=True)
    
    em_embeddings = Reshape((max_sequence_length, 1))
    
    q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match),])
    q1_encoded = SpatialDropout1D(0.2)(q1_embed)
    
    q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match),])
    q2_encoded = SpatialDropout1D(0.2)(q2_embed)


    #capsule_pooling = Capsule(num_capsule=3, dim_capsule=600, routings=2, share_weights=True)
    nb_filters = 64
    
    cnns = [Conv1D(64, 1, strides=1, padding='same', activation='relu') for i in range(3)]
    gates_cnns = [Conv1D(nb_filters, 3, dilation_rate=1, padding='same', activation='tanh') for i in range(3)]
    sigm_cnns = [Conv1D(nb_filters, 3, dilation_rate=1, padding='same', activation='sigmoid') for i in range(3)]
    
    for i in range(len(cnns)):
        drop = Dropout(0.1)
        q1_t = gates_cnns[i](q1_encoded)
        q2_t = gates_cnns[i](q2_encoded)    
        
        q1_s = sigm_cnns[i](q1_encoded)
        q2_s = sigm_cnns[i](q2_encoded)        
        
        q1_x = Multiply()([q1_t, q1_s])
        q1_x = cnns[i](q1_x)
        q1_x = drop(q1_x)
        
        q2_x = Multiply()([q2_t, q2_s])
        q2_x = cnns[i](q2_x)
        q2_x = drop(q2_x)

        q1_aligned, q2_aligned = soft_attention_alignment(q1_x, q2_x)
        q1_encoded = Concatenate()([q1_x, q2_aligned, q1_encoded])
        q2_encoded = Concatenate()([q2_x, q1_aligned, q2_encoded]) 
    
    #capsule_pooling = Capsule(num_capsule=3, dim_capsule=600, routings=2, share_weights=True)
    
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    
    attn = AttentionWeightedAverage()
    
    
    q1_rep = apply_multiple(q1_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])
    q2_rep = apply_multiple(q2_encoded, [GlobalAvgPool1D(), GlobalMaxPool1D(), attn])    
    
    
    #meta_features = BatchNormalization()(meta_features_input)
    #meta_features = Dropout(0.8)(meta_features)
    #meta_features = Highway(activation='relu')(meta_features)
    # Pooling
    #q1_rep = Flatten()(capsule_pooling(q1_encoded))
    #q2_rep = Flatten()(capsule_pooling(q2_encoded))
    #drops = Dropout(0.3)
    #q1_rep = shrink(drops(q1_rep))
    #q2_rep = shrink(drops(q2_rep))
    #meta_features = BatchNormalization()(meta_features_input)
    #meta_features = Dropout(0.8)(meta_features)
    #meta_features = Highway(activation='relu')(meta_features)
    
    # Classifier
    q_diff = substract(q1_rep, q2_rep)
    q_multi = Multiply()([q1_rep, q2_rep])
    h_all = Concatenate()([q1_rep, q2_rep, q_diff, q_multi,])
    h_all = Dropout(0.2)(h_all)
    h_all = BatchNormalization()(h_all)
    #h_all = Dropout(0.35)(h_all)
    #h_all = Highway(activation='relu')(h_all)
    #h_all = Highway(activation='relu')(h_all)    
    h_all = Dense(64, activation='relu', kernel_initializer='glorot_uniform', kernel_regularizer=regularizers.l2(1e-6))(h_all)
    out_ = Dense(3, activation='softmax')(h_all)

    model = Model(inputs=[q1, q2, meta_features_input, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [26]:
fold_count = 8
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_dense_cnn(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=1024,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=25)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "3Embedding-3LayersDenseCNN42-NoDrop-NoClassWeighted-withEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2][:, -1],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
first_exact_match (InputLayer)  (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
second_exact_match (InputLayer) (None, 50)           0                                            
_____________________________________________________________________________________________

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 64s 228us/step - loss: 0.4104 - acc: 0.8121 - weighted_accuracy: 0.7840 - val_loss: 0.3446 - val_acc: 0.8413 - val_weighted_accuracy: 0.8235
Epoch 2/500
280483/280483 [==============================] - 62s 222us/step - loss: 0.3270 - acc: 0.8504 - weighted_accuracy: 0.8256 - val_loss: 0.3129 - val_acc: 0.8562 - val_weighted_accuracy: 0.8316
Epoch 3/500
280483/280483 [==============================] - 61s 216us/step - loss: 0.3045 - acc: 0.8634 - weighted_accuracy: 0.8399 - val_loss: 0.3083 - val_acc: 0.8614 - val_weighted_accuracy: 0.8374
Epoch 4/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.2914 - acc: 0.8692 - weighted_accuracy: 0.8460 - val_loss: 0.3200 - val_acc: 0.8530 - val_weighted_accuracy: 0.8371
Epoch 5/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.2819 - acc: 0.8740 - weighted_accuracy: 0.8515 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 62s 220us/step - loss: 0.4205 - acc: 0.8097 - weighted_accuracy: 0.7804 - val_loss: 0.3354 - val_acc: 0.8391 - val_weighted_accuracy: 0.8200
Epoch 2/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.3283 - acc: 0.8496 - weighted_accuracy: 0.8236 - val_loss: 0.3025 - val_acc: 0.8623 - val_weighted_accuracy: 0.8404
Epoch 3/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3074 - acc: 0.8610 - weighted_accuracy: 0.8366 - val_loss: 0.2995 - val_acc: 0.8638 - val_weighted_accuracy: 0.8452
Epoch 4/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2935 - acc: 0.8682 - weighted_accuracy: 0.8443 - val_loss: 0.2891 - val_acc: 0.8702 - val_weighted_accuracy: 0.8466
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2851 - acc: 0.8728 - weighted_accuracy: 0.8498 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 62s 221us/step - loss: 0.4006 - acc: 0.8167 - weighted_accuracy: 0.7872 - val_loss: 0.3592 - val_acc: 0.8246 - val_weighted_accuracy: 0.8136
Epoch 2/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.3235 - acc: 0.8527 - weighted_accuracy: 0.8265 - val_loss: 0.3403 - val_acc: 0.8381 - val_weighted_accuracy: 0.8267
Epoch 3/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.3024 - acc: 0.8635 - weighted_accuracy: 0.8387 - val_loss: 0.3194 - val_acc: 0.8513 - val_weighted_accuracy: 0.8354
Epoch 4/500
280483/280483 [==============================] - 60s 214us/step - loss: 0.2903 - acc: 0.8704 - weighted_accuracy: 0.8466 - val_loss: 0.3132 - val_acc: 0.8576 - val_weighted_accuracy: 0.8354
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2805 - acc: 0.8751 - weighted_accuracy: 0.8513 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 62s 222us/step - loss: 0.3939 - acc: 0.8179 - weighted_accuracy: 0.7885 - val_loss: 0.3590 - val_acc: 0.8289 - val_weighted_accuracy: 0.8113
Epoch 2/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3248 - acc: 0.8517 - weighted_accuracy: 0.8260 - val_loss: 0.3341 - val_acc: 0.8466 - val_weighted_accuracy: 0.8315
Epoch 3/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3029 - acc: 0.8635 - weighted_accuracy: 0.8388 - val_loss: 0.3239 - val_acc: 0.8493 - val_weighted_accuracy: 0.8315
Epoch 4/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2900 - acc: 0.8707 - weighted_accuracy: 0.8472 - val_loss: 0.3163 - val_acc: 0.8549 - val_weighted_accuracy: 0.8392
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2808 - acc: 0.8751 - weighted_accuracy: 0.8526 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 62s 222us/step - loss: 0.4116 - acc: 0.8128 - weighted_accuracy: 0.7848 - val_loss: 0.3754 - val_acc: 0.8139 - val_weighted_accuracy: 0.7970
Epoch 2/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3265 - acc: 0.8505 - weighted_accuracy: 0.8247 - val_loss: 0.3446 - val_acc: 0.8340 - val_weighted_accuracy: 0.8074
Epoch 3/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3036 - acc: 0.8629 - weighted_accuracy: 0.8380 - val_loss: 0.3414 - val_acc: 0.8362 - val_weighted_accuracy: 0.8208
Epoch 4/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2893 - acc: 0.8707 - weighted_accuracy: 0.8467 - val_loss: 0.3226 - val_acc: 0.8492 - val_weighted_accuracy: 0.8255
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2808 - acc: 0.8756 - weighted_accuracy: 0.8522 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 63s 224us/step - loss: 0.3977 - acc: 0.8170 - weighted_accuracy: 0.7882 - val_loss: 0.3939 - val_acc: 0.8075 - val_weighted_accuracy: 0.7779
Epoch 2/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3201 - acc: 0.8532 - weighted_accuracy: 0.8278 - val_loss: 0.3539 - val_acc: 0.8328 - val_weighted_accuracy: 0.8039
Epoch 3/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3004 - acc: 0.8640 - weighted_accuracy: 0.8395 - val_loss: 0.3544 - val_acc: 0.8340 - val_weighted_accuracy: 0.8016
Epoch 4/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2874 - acc: 0.8706 - weighted_accuracy: 0.8471 - val_loss: 0.3475 - val_acc: 0.8424 - val_weighted_accuracy: 0.8065
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2778 - acc: 0.8762 - weighted_accuracy: 0.8536 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 63s 224us/step - loss: 0.4101 - acc: 0.8123 - weighted_accuracy: 0.7831 - val_loss: 0.3517 - val_acc: 0.8357 - val_weighted_accuracy: 0.8177
Epoch 2/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3264 - acc: 0.8506 - weighted_accuracy: 0.8249 - val_loss: 0.3172 - val_acc: 0.8543 - val_weighted_accuracy: 0.8306
Epoch 3/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3040 - acc: 0.8624 - weighted_accuracy: 0.8379 - val_loss: 0.3152 - val_acc: 0.8574 - val_weighted_accuracy: 0.8347
Epoch 4/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2920 - acc: 0.8680 - weighted_accuracy: 0.8441 - val_loss: 0.3098 - val_acc: 0.8595 - val_weighted_accuracy: 0.8366
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2823 - acc: 0.8741 - weighted_accuracy: 0.8509 - val_lo

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 63s 225us/step - loss: 0.4110 - acc: 0.8116 - weighted_accuracy: 0.7838 - val_loss: 0.3382 - val_acc: 0.8397 - val_weighted_accuracy: 0.8160
Epoch 2/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3278 - acc: 0.8501 - weighted_accuracy: 0.8250 - val_loss: 0.3208 - val_acc: 0.8526 - val_weighted_accuracy: 0.8312
Epoch 3/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.3061 - acc: 0.8615 - weighted_accuracy: 0.8377 - val_loss: 0.3009 - val_acc: 0.8634 - val_weighted_accuracy: 0.8378
Epoch 4/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2934 - acc: 0.8685 - weighted_accuracy: 0.8456 - val_loss: 0.2974 - val_acc: 0.8650 - val_weighted_accuracy: 0.8375
Epoch 5/500
280483/280483 [==============================] - 60s 215us/step - loss: 0.2847 - acc: 0.8735 - weighted_accuracy: 0.8508 - val_lo

# ESIM

In [47]:
from keras import regularizers

def get_ESIM(nb_words, embedding_dim, embedding_matrix, max_sequence_length, out_size,
    projection_dim=50, projection_hidden=0, projection_dropout=0.2,
    compare_dim=288, compare_dropout=0.2,
    dense_dim=50, dense_dropout=0.2,
    lr=1e-3, activation='relu'):

    q1 = Input(shape=(max_sequence_length,), name='first_sentences')
    q2 = Input(shape=(max_sequence_length,), name='second_sentences')
    q1_exact_match = Input(shape=(max_sequence_length,), name='first_exact_match')
    q2_exact_match = Input(shape=(max_sequence_length,), name='second_exact_match')
    
    input_layer_3 = Input(shape=(36,), name='mata-features', dtype="float32")

    #input_encoded = BatchNormalization()(input_layer_3)
    input_encoded = Dense(2016, activation='elu')(input_layer_3)
    input_encoded = Dropout(0.25)(input_encoded)
    
    embedding = Embedding(nb_words, 150,
                          weights=[embedding_matrix],
                          input_length=max_sequence_length,
                          trainable=False)
 
    em_embeddings = Embedding(2, 1,
                     input_length=max_sequence_length,
                     trainable=True)   
    
    #q1_embed = Concatenate()([embedding(q1), em_embeddings(q1_exact_match)])
    q1_embed = embedding(q1)
    q1_embed = SpatialDropout1D(0.1)(q1_embed)
    
    #q2_embed = Concatenate()([embedding(q2), em_embeddings(q2_exact_match)])
    q2_embed = embedding(q2)
    q2_embed = SpatialDropout1D(0.1)(q2_embed)

    batch_norm = BatchNormalization(axis=-1)
    q1_embed = batch_norm(q1_embed)
    q2_embed = batch_norm(q2_embed)
    
    aggreation_gru = Bidirectional(CuDNNLSTM(72, return_sequences=True))
 
    q1_seq = aggreation_gru(q1_embed)
    q2_seq = aggreation_gru(q2_embed)
        
    q1_aligned, q2_aligned = soft_attention_alignment(q1_seq, q2_seq)
    q1_vec = Concatenate()([q1_seq, q2_aligned, substract(q1_seq, q2_aligned), Multiply()([q1_seq, q2_aligned])])
    q2_vec = Concatenate()([q2_seq, q1_aligned, substract(q2_seq, q1_aligned), Multiply()([q2_seq, q1_aligned])])
    
    compare_gru = Bidirectional(CuDNNLSTM(72, return_sequences=True))
    
    q1_rep = compare_gru(q1_vec)
    q2_rep = compare_gru(q2_vec)
    
    q1_rep = apply_multiple(q1_rep, [GlobalAvgPool1D(), GlobalMaxPool1D()])
    q2_rep = apply_multiple(q2_rep, [GlobalAvgPool1D(), GlobalMaxPool1D()])    
    
    h_all = Concatenate()([q1_rep, q2_rep])
    h_all = BatchNormalization()(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = BatchNormalization()(h_all)
    h_all = Dropout(0.2)(h_all)
    
    h_all = Dense(256, activation='elu')(h_all)
    h_all = BatchNormalization()(h_all)
    h_all = Dropout(0.2)(h_all)
    
    out_ = Dense(3, activation='softmax')(h_all)
    
    model = Model(inputs=[q1, q2, input_layer_3, q1_exact_match, q2_exact_match], outputs=out_)
    model.compile(optimizer=Adam(lr=lr, decay=1e-6, clipnorm=1.5, amsgrad=True), loss='categorical_crossentropy',
    metrics=['accuracy', weighted_accuracy])
    model.summary()
    return model

In [48]:
fold_count = 8
embedding_matrix=meta_embeddings

for i in range(4, len(model_manager.models_tag)):
    print("Work on model", i)
    model_tag = model_manager.models_tag[i]
    model_func = model_manager.model_funcs[i]
    #models_checkpoints_path = model_manager.models_checkpoints_pathes[i]
    models_checkpoints_path = "3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM"

    model_submit_prefix = model_manager.submit_predix[i]
    model_class_weights = model_manager.model_class_weights[i]
    model_class_weights = None
    model_scale_sample_weights = model_manager.model_scale_sample_weights[i]
    model_patiences = model_manager.model_patiences[i]
    
    def _agent_get_model():
        return get_ESIM(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
        return model_func(NB_WORDS, EMBEDDING_DIM, embedding_matrix, MAX_SEQUENCE_LENGTH, OUT_SIZE)
    
    test_predicts_list = []
    oofs_predictions = []
    pre_trained_models = []
        
    trainer = KerasModelTrainer(model_stamp=models_checkpoints_path, epoch_num=500)
    models, score, folds_preds = trainer.train_folds(X=trains, y=labels, augments=None, fold_count=fold_count, batch_size=128,
        em_train_features=em_train_features,
        scale_sample_weight=model_scale_sample_weights, class_weight=model_class_weights,
        get_model_func=_agent_get_model, 
        patience=10)

    print("score", score)
    oofs_dir = "../data/oofs/"
    output_dir = "../data/output/"
    onehot_pred_dir = "../data/one_hot_pred/"

    model_submit_prefix = "3Embedding-ESIM-Drop01-NoMeta-NoClassWeighted-NoEM"
    
    oofs_path = oofs_dir + model_submit_prefix
    output_path = output_dir + model_submit_prefix
    one_hot_pred_path = onehot_pred_dir + "One-Hot" + model_submit_prefix

    print("Predicting training results...")
    train_predicts = np.concatenate(folds_preds, axis=0)
    oofs = pd.DataFrame({"unrelated": train_predicts[:, 0], "agreed": train_predicts[:, 1], "disagreed": train_predicts[:, 2]})
    submit_path = oofs_path + "-Train-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    oofs.to_csv(submit_path, index=False)

    print("Predicting testing results...")
    test_predicts_list = []
    for fold_id, model in enumerate(models):
        test_predicts = model.predict({"first_sentences":tests[0],
                                       "second_sentences":tests[1],
                                       "mata-features":tests[2],
                                       "first_exact_match": em_test_features[0],
                                       "second_exact_match": em_test_features[1],
                                      }, batch_size=128, verbose=1)
        test_predicts_list.append(test_predicts)

    test_predicts = np.zeros(test_predicts_list[0].shape)
    for fold_predict in test_predicts_list:
        test_predicts += fold_predict
    test_predicts /= len(test_predicts_list)

    test_predicts = pd.DataFrame({"unrelated": test_predicts[:, 0], "agreed": test_predicts[:, 1], "disagreed": test_predicts[:, 2]})
    submit_path = output_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    test_predicts.to_csv(submit_path, index=False) # 0.3343
    
    print("Predicting labeled testing results...")
    ids = pd.read_csv("../data/dataset/test.csv")
    pred_labels = test_predicts.idxmax(axis=1)
    sub = pd.DataFrame({"Id": ids['id'].values, "Category": pred_labels})
    submit_path = one_hot_pred_path + "-L{:4f}-NB{:d}.csv".format(score, NB_WORDS)
    sub = sub.to_csv(submit_path, index=False)
    break

Work on model 4
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_sentences (InputLayer)    (None, 50)           0                                            
__________________________________________________________________________________________________
second_sentences (InputLayer)   (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_19 (Embedding)        (None, 50, 150)      750000      first_sentences[0][0]            
                                                                 second_sentences[0][0]           
__________________________________________________________________________________________________
spatial_dropout1d_17 (SpatialDr (None, 50, 150)      0           embedding_19[0][0]          

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 251s 894us/step - loss: 0.3533 - acc: 0.8434 - weighted_accuracy: 0.8143 - val_loss: 0.3416 - val_acc: 0.8421 - val_weighted_accuracy: 0.8225
Epoch 2/500
280483/280483 [==============================] - 220s 786us/step - loss: 0.2766 - acc: 0.8787 - weighted_accuracy: 0.8565 - val_loss: 0.2930 - val_acc: 0.8692 - val_weighted_accuracy: 0.8501
Epoch 3/500
280483/280483 [==============================] - 220s 785us/step - loss: 0.2533 - acc: 0.8909 - weighted_accuracy: 0.8705 - val_loss: 0.3151 - val_acc: 0.8620 - val_weighted_accuracy: 0.8483
Epoch 4/500
280483/280483 [==============================] - 218s 779us/step - loss: 0.2371 - acc: 0.8989 - weighted_accuracy: 0.8800 - val_loss: 0.3167 - val_acc: 0.8625 - val_weighted_accuracy: 0.8527
Epoch 5/500
280483/280483 [==============================] - 218s 779us/step - loss: 0.2245 - acc: 0.9048 - weighted_accuracy: 0.8865 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 251s 896us/step - loss: 0.3591 - acc: 0.8409 - weighted_accuracy: 0.8119 - val_loss: 0.3082 - val_acc: 0.8619 - val_weighted_accuracy: 0.8438
Epoch 2/500
280483/280483 [==============================] - 234s 833us/step - loss: 0.2801 - acc: 0.8770 - weighted_accuracy: 0.8548 - val_loss: 0.2799 - val_acc: 0.8766 - val_weighted_accuracy: 0.8581
Epoch 3/500
280483/280483 [==============================] - 219s 782us/step - loss: 0.2561 - acc: 0.8893 - weighted_accuracy: 0.8685 - val_loss: 0.2781 - val_acc: 0.8786 - val_weighted_accuracy: 0.8580
Epoch 4/500
280483/280483 [==============================] - 220s 783us/step - loss: 0.2389 - acc: 0.8972 - weighted_accuracy: 0.8778 - val_loss: 0.2722 - val_acc: 0.8817 - val_weighted_accuracy: 0.8652
Epoch 5/500
280483/280483 [==============================] - 219s 780us/step - loss: 0.2270 - acc: 0.9035 - weighted_accuracy: 0.8851 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 253s 903us/step - loss: 0.3538 - acc: 0.8438 - weighted_accuracy: 0.8142 - val_loss: 0.3376 - val_acc: 0.8444 - val_weighted_accuracy: 0.8347
Epoch 2/500
280483/280483 [==============================] - 229s 815us/step - loss: 0.2761 - acc: 0.8787 - weighted_accuracy: 0.8559 - val_loss: 0.3302 - val_acc: 0.8460 - val_weighted_accuracy: 0.8331
Epoch 3/500
280483/280483 [==============================] - 232s 825us/step - loss: 0.2531 - acc: 0.8906 - weighted_accuracy: 0.8693 - val_loss: 0.2969 - val_acc: 0.8683 - val_weighted_accuracy: 0.8522
Epoch 4/500
280483/280483 [==============================] - 231s 823us/step - loss: 0.2380 - acc: 0.8983 - weighted_accuracy: 0.8785 - val_loss: 0.2997 - val_acc: 0.8673 - val_weighted_accuracy: 0.8548
Epoch 5/500
280483/280483 [==============================] - 244s 869us/step - loss: 0.2246 - acc: 0.9046 - weighted_accuracy: 0.8862 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 230s 820us/step - loss: 0.3545 - acc: 0.8431 - weighted_accuracy: 0.8134 - val_loss: 0.3102 - val_acc: 0.8575 - val_weighted_accuracy: 0.8371
Epoch 2/500
280483/280483 [==============================] - 224s 798us/step - loss: 0.2760 - acc: 0.8789 - weighted_accuracy: 0.8562 - val_loss: 0.2983 - val_acc: 0.8642 - val_weighted_accuracy: 0.8503
Epoch 3/500
280483/280483 [==============================] - 223s 794us/step - loss: 0.2534 - acc: 0.8907 - weighted_accuracy: 0.8699 - val_loss: 0.2934 - val_acc: 0.8686 - val_weighted_accuracy: 0.8579
Epoch 4/500
280483/280483 [==============================] - 224s 797us/step - loss: 0.2377 - acc: 0.8986 - weighted_accuracy: 0.8791 - val_loss: 0.3054 - val_acc: 0.8635 - val_weighted_accuracy: 0.8561
Epoch 5/500
280483/280483 [==============================] - 250s 891us/step - loss: 0.2258 - acc: 0.9042 - weighted_accuracy: 0.8857 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 238s 849us/step - loss: 0.3518 - acc: 0.8446 - weighted_accuracy: 0.8151 - val_loss: 0.3146 - val_acc: 0.8557 - val_weighted_accuracy: 0.8377
Epoch 2/500
280483/280483 [==============================] - 250s 893us/step - loss: 0.2738 - acc: 0.8799 - weighted_accuracy: 0.8567 - val_loss: 0.3123 - val_acc: 0.8621 - val_weighted_accuracy: 0.8425
Epoch 3/500
280483/280483 [==============================] - 241s 859us/step - loss: 0.2518 - acc: 0.8921 - weighted_accuracy: 0.8709 - val_loss: 0.3104 - val_acc: 0.8624 - val_weighted_accuracy: 0.8505
Epoch 4/500
280483/280483 [==============================] - 247s 880us/step - loss: 0.2350 - acc: 0.8997 - weighted_accuracy: 0.8803 - val_loss: 0.3070 - val_acc: 0.8630 - val_weighted_accuracy: 0.8480
Epoch 5/500
280483/280483 [==============================] - 251s 894us/step - loss: 0.2227 - acc: 0.9057 - weighted_accuracy: 0.8876 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 223s 794us/step - loss: 0.3520 - acc: 0.8444 - weighted_accuracy: 0.8156 - val_loss: 0.3368 - val_acc: 0.8457 - val_weighted_accuracy: 0.8130
Epoch 2/500
280483/280483 [==============================] - 220s 783us/step - loss: 0.2737 - acc: 0.8805 - weighted_accuracy: 0.8582 - val_loss: 0.3194 - val_acc: 0.8528 - val_weighted_accuracy: 0.8239
Epoch 3/500
280483/280483 [==============================] - 246s 878us/step - loss: 0.2513 - acc: 0.8923 - weighted_accuracy: 0.8716 - val_loss: 0.3269 - val_acc: 0.8529 - val_weighted_accuracy: 0.8212
Epoch 4/500
280483/280483 [==============================] - 241s 858us/step - loss: 0.2350 - acc: 0.8994 - weighted_accuracy: 0.8802 - val_loss: 0.3165 - val_acc: 0.8584 - val_weighted_accuracy: 0.8358
Epoch 5/500
280483/280483 [==============================] - 223s 796us/step - loss: 0.2222 - acc: 0.9059 - weighted_accuracy: 0.8878 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 247s 881us/step - loss: 0.3536 - acc: 0.8436 - weighted_accuracy: 0.8141 - val_loss: 0.3051 - val_acc: 0.8626 - val_weighted_accuracy: 0.8504
Epoch 2/500
280483/280483 [==============================] - 247s 880us/step - loss: 0.2759 - acc: 0.8791 - weighted_accuracy: 0.8563 - val_loss: 0.2915 - val_acc: 0.8681 - val_weighted_accuracy: 0.8413
Epoch 3/500
280483/280483 [==============================] - 231s 823us/step - loss: 0.2538 - acc: 0.8908 - weighted_accuracy: 0.8700 - val_loss: 0.2859 - val_acc: 0.8722 - val_weighted_accuracy: 0.8571
Epoch 4/500
280483/280483 [==============================] - 223s 796us/step - loss: 0.2372 - acc: 0.8986 - weighted_accuracy: 0.8790 - val_loss: 0.3022 - val_acc: 0.8709 - val_weighted_accuracy: 0.8520
Epoch 5/500
280483/280483 [==============================] - 223s 795us/step - loss: 0.2254 - acc: 0.9044 - weighted_accuracy: 0.8863 - v

Train on 280483 samples, validate on 40069 samples
Epoch 1/500
280483/280483 [==============================] - 256s 914us/step - loss: 0.3555 - acc: 0.8428 - weighted_accuracy: 0.8137 - val_loss: 0.3123 - val_acc: 0.8631 - val_weighted_accuracy: 0.8400
Epoch 2/500
280483/280483 [==============================] - 240s 856us/step - loss: 0.2769 - acc: 0.8788 - weighted_accuracy: 0.8567 - val_loss: 0.2823 - val_acc: 0.8714 - val_weighted_accuracy: 0.8486
Epoch 3/500
280483/280483 [==============================] - 223s 795us/step - loss: 0.2532 - acc: 0.8913 - weighted_accuracy: 0.8711 - val_loss: 0.2858 - val_acc: 0.8751 - val_weighted_accuracy: 0.8569
Epoch 4/500
280483/280483 [==============================] - 223s 796us/step - loss: 0.2373 - acc: 0.8990 - weighted_accuracy: 0.8804 - val_loss: 0.2815 - val_acc: 0.8748 - val_weighted_accuracy: 0.8592
Epoch 5/500
280483/280483 [==============================] - 223s 795us/step - loss: 0.2250 - acc: 0.9050 - weighted_accuracy: 0.8876 - v